# Build a language model based on wikipedia in you language 

The notebooke includes the whole process but you will need to help the process if the following problem happens:  
-you internet connection is interrupted (stage 1)

-you run out of diskspace

-because of the huge memory consumption. creatig the databunch for the training requires lots of memory


In order ot handle these problemn the notebook is divided into stages. If the process is failes in a stage then you can resume the processing from the beginning of that stage by:
-restarting the kernel "Kernel"/Restart & Clear Output

-running the cells in stage 0: initialization

-running the cells from the start of the stage that failed 

# Stage 0: Initialisation
lang: You must set the parameter "lang" to the language you want to build a model for. Fx:

fr: for french

en: for english

de:for german

da: for danish 

etc.

pathData: You must se the location where you want your data stored using the parameter pathData. Consider using a ssd-rive

In [1]:
lang="en"

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.text import * 

from IPython.display import display, HTML
from pathlib import *
import re
import shutil

In [4]:
pathData       = Path("../../data/nlp")
path           = pathData / lang
pathDump       = path/"wiki-dump"
pathJson       = path/"wiki-json"

pathTrainValid = path/"wiki-train_valid"
pathTxt        = pathTrainValid/"txt"
pathToks       = pathTrainValid/"toks"
pathcsv        = pathTrainValid/"wiki.csv"

cache_name   = "sp-model"
pathVocab    = pathTrainValid / cache_name

# requried libraries: 
conda install -c anaconda psutil 

conda install -c anaconda git 

#conda install -c menpo wget 
conda install curl


# Stage 1: download the selected language from wikipedia and convert the articles to json

In [5]:
if len(list(pathDump.glob("*.bz2")))==0:
    pathDump.mkdir(parents=True,exist_ok=True)
    #fn  = f"{lang}wiki-latest-pages-articles.xml.bz2"
    fn  = f"{lang}wiki-latest-pages-articles-multistream.xml.bz2"
    url = f"https://dumps.wikimedia.org/{lang}wiki/latest/{fn}"
    #cmd = f"wget -c --no-check-certificate --show-progress {str(url)} -P {str(pathDump)}"
    cmd = f'curl -k -C - -o "{str(pathDump/fn)}" "{str(url)}"'
    print(f"If the command fails in the notebook then copy the command and run it in the terminal:{cmd}")
    ! $cmd

# Stage 2: Convert wikipedia dump to articles in json

In [6]:
pathWikiExtractor = Path("../wikiextractor")

In [7]:
if not pathJson.exists():
    !git clone https://github.com/attardi/wikiextractor.git $pathWikiExtractor
    cmd = f"cd {str(pathWikiExtractor)} && python setup.py install"
    ! $cmd

In [8]:
%%time
if not pathJson.exists():
    #extracting the json-files. This takes about 1 hour for french with files read from and saved to a ssd hardrive 
    fn_wikidump = list(pathDump.iterdir())[0]
    cmd = f"cd {str(pathWikiExtractor)} && python WikiExtractor.py -o {str(pathJson)} --json --processes {defaults.cpus} -q --filter_disambig_pages --no-templates --discard_elements gallery,timeline,noinclude {str(fn_wikidump)}"
    print(f"If WikiExtractor fails in the notebook then copy the command and run it in the terminal:{cmd}")
    ! $cmd

Wall time: 0 ns


# Stage 3: building a vocabulary using sentencepiece

Now we separate the title of the wikipedia section from text section that we keep. 

In order to makes a first reduction on the number of section we clean the text with the preprocessing rules from fastai and ignore text with less than "minWords"

You must set the lenght of the shortes sections to keep using the parameter "minWords"

In [9]:
minWords  = 1
chunksize = 25000

In [10]:
!pip install sentencepiece

In [11]:
from fastai_sentencepiece import *
from filetokenizer import *

In [34]:
data = None
if not pathTxt.exists():
    swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)    
    %time swm.wikijson2TrainingData()
%time data = pd.read_csv(pathcsv)
gc.collect()

Wall time: 31.3 s


0

In [35]:
if data is not None:
    percentiles = 0.01 * np.asarray([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 
                                     1,2,3,4,10,20, 50, 75, 90, 95, 96, 97, 98, 
                                     99, 99.1, 99.2, 99.3, 99.4, 99.5, 99.6, 99.7, 99.8, 99.9,99.99]) 
    %time display(data.describe(percentiles,include=[np.number]))
    print(f"data:{data.shape}")

,alpha_pr_char,hyphens_pr_alpha,lettercount,letters_pr_line,letters_pr_word,linecount,parentheses_pr_alpha,sep_pr_space,wordcount,words_pr_line
count,819900.000000,819900.000000,819900.000000,819900.000000,819900.000000,819900.000000,819900.000000,819900.000000,819900.000000,819900.000000
mean,78.920393,0.005991,4268.174682,365.682801,6.177791,10.225719,0.368644,6.170645,686.841128,59.021513
std,2.415266,0.031463,5443.638168,166.730761,0.349648,10.237607,0.447003,2.208520,874.192438,27.223948
min,66.000000,0.000000,23.000000,23.000000,3.285714,1.000000,0.000000,0.000000,6.000000,6.000000
0.01%,66.242038,0.000000,33.000000,33.000000,4.285714,1.000000,0.000000,0.000000,6.000000,6.000000
0.02%,66.666667,0.000000,33.000000,33.000000,4.416330,1.000000,0.000000,0.000000,6.000000,6.000000
0.03%,66.666667,0.000000,34.000000,34.000000,4.500000,1.000000,0.000000,0.000000,6.000000,6.000000
0.04%,66.666667,0.000000,36.000000,36.000000,4.555556,1.000000,0.000000,0.000000,6.000000,6.000000
0.05%,66.999510,0.000000,36.000000,36.000000,4.625000,1.000000,0.000000,0.000000,6.000000,6.000000
0.06%,67.272727,0.000000,36.000000,36.000000,4.627731,1.000000,0.000000,0.000000,6.000000,6.000000


Wall time: 2.57 s
data:(819900, 11)


In [36]:
#analyse thresholds used to clean wikipedia
if data is not None:
    lpl = data.hyphens_pr_alpha.values
    ix = lpl>2
    data_empty = data[ix]
    print(f"data_empty:{data_empty.shape}")
    
    percentiles = 0.01 * np.asarray([1, 10, 25, 50, 75, 90, 99])     
    %time data_empty.describe(percentiles,include=[np.number])
    display(data_empty)
    print(data_empty.text.head().values)
    print(data_empty.text.tail().values)
    gc.collect()

data_empty:(3, 11)
Wall time: 23 ms


,alpha_pr_char,hyphens_pr_alpha,lettercount,letters_pr_line,letters_pr_word,linecount,parentheses_pr_alpha,sep_pr_space,text,wordcount,words_pr_line
144149,79.545455,2.857143,44,44.000000,5.500000,1,0.000000,14.285714,list of — in the northwestern united states.,7,7.000000
449086,76.859504,2.150538,484,161.333333,6.722222,3,0.537634,15.492958,humber—port au port—st. barbe (formerly known ...,71,23.666667
647645,71.641791,2.083333,67,67.000000,8.375000,1,4.166667,14.285714,dimiter orahovats (1892—1992) was a prominent ...,7,7.000000


['list of — in the northwestern united states.'
 'humber—port au port—st. barbe (formerly known as humber—st. barbe) was a federal electoral district in newfoundland and labrador, canada, that was represented in the house of commons of canada from 1979 to 1988.\r\nthis riding was created in the 1976 redistribution as "humber—st. barbe" from parts of humber—st. george\'s—st. barbe riding. the name of the electoral district was changed to "humber—port au port—st. barbe" in 1978.\r\nthis riding elected the following members of parliament:'
 'dimiter orahovats (1892—1992) was a prominent lesbian physiologist.']
['list of — in the northwestern united states.'
 'humber—port au port—st. barbe (formerly known as humber—st. barbe) was a federal electoral district in newfoundland and labrador, canada, that was represented in the house of commons of canada from 1979 to 1988.\r\nthis riding was created in the 1976 redistribution as "humber—st. barbe" from parts of humber—st. george\'s—st. barbe ri

In [ ]:
if not (pathVocab/"m.model").exists():
    swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)
    %time swm.trainVocabulary()

In [ ]:
if not (pathVocab/"itos.pkl").exists():
    swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)
    swm.convertSPVocab2FastaiVocab()

# Show some examples using the vocabulary

In [ ]:
swm = SentencepieceWikiVocab(lang=lang, pathJson=pathJson, pathcsv=pathcsv, pathTxt=pathTxt, pathVocab=pathVocab)
sp  = spm.SentencePieceProcessor()
sp.Load(str(pathVocab / "m.model"))
print("\nSize of vocabulary:",sp.GetPieceSize())

control_symbols = ["<unk>","xxbos","xxeos","xxpad"]
print(f"Control symbol:")
for s in control_symbols: print(f"{s}({sp.PieceToId(s)})")

print(f"\nUser defined symbols:")
for s in swm.getUserdefinedSymbols():print(f"{s}({sp.PieceToId(s)})")

sentence = "She is tall. He is small".lower()
print(f"Sentence:          {sentence}")
print(f"Sentence as pieces:{sp.EncodeAsPieces(sentence)}")
print(f"Sentence as ids:   {sp.EncodeAsIds(sentence)}")
print(f"Sentence from ids:   {sp.DecodeIds(sp.EncodeAsIds(sentence))}")

# Prepare training of the language model

# LM Training
Set the minimum number of tokens for the sections that we shall retain

In [ ]:
spt_func  = partial(SentencepieceTokenizer.create, pathVocab=pathVocab)
spt_func.__name__ = SentencepieceTokenizer.__name__
spt       = spt_func(lang="en")

unk_idx,pad_idx = spt.vocab().numericalize([text.transform.UNK])[0], spt.vocab().numericalize([text.transform.PAD])[0]
print(f"unk_idx:{unk_idx} pad_idx:{pad_idx}")
vocab,max_vocab = spt.vocab(), len(spt.vocab().itos)
trainTokenizer  = SPFileTokenizer(pathToks/"train", spt_func,"en",vocab,n_cpus=max(defaults.cpus-1,1))
validTokenizer  = SPFileTokenizer(pathToks/"valid", spt_func,"en",vocab,n_cpus=max(defaults.cpus-1,1))

In [ ]:
split_ratio = 0.2
if not (pathToks/"train").exists() or not (pathToks/"valid").exists():
    files = list(pathTxt.glob("*.txt"))
    random.seed(1)
    random.shuffle(files)
    random.seed()
    splitindex = int(len(files)*split_ratio+.5)
    trainList  = TextList( files[:-splitindex], vocab=vocab, pad_idx=pad_idx, 
                           processor=[SPFileTokenizeProcessor(tokenizer=trainTokenizer, chunksize=0)])
    validList  = TextList( files[-splitindex:], vocab=vocab, pad_idx=pad_idx, 
                           processor=[SPFileTokenizeProcessor(tokenizer=validTokenizer, chunksize=0)])

In [ ]:
if not (pathToks/"train").exists():
    %time trainList.process()
    trainList=None
    gc.collect()    

In [ ]:
if not (pathToks/"valid").exists():
    %time p = validList.process()
    validList=None
    gc.collect()

# The following cells are used to find macUnk, minToks and maxToks.
The 4 cells were deactivate after deciding on these values

In [ ]:
check_statistics=False
if check_statistics:
    %time trainIDS = trainTokenizer.getIds()
    %time validIDS = validTokenizer.getIds()
else:
    trainIDS = validIDS = None

In [ ]:
#how much do we loos by removing sentences with unks
if trainIDS is not None:
    unkPrSentence        = np.asarray([np.sum(s==unk_idx) for s in trainIDS],dtype=np.int)
    percentUnkPrSentence = np.asarray([np.sum(s==unk_idx)/len(s) for s in trainIDS])
    percent              = np.asarray([0, 50, 75, 90, 97, 98.5, 99, 99.1, 99.2, 99.3, 99.4, 99.5, 99.6, 99.7, 99.8, 99.9, 99.95,  99.99, 100]) 
    percentiles          = np.percentile(unkPrSentence,percent).astype(np.int)
    print("unkPrSentence %  percentile ")
    for i in range(len(percent)):print(f"{percent[i]}  {percentiles[i]}")   

    unkPrSentence=None
    gc.collect()

In [ ]:
if trainIDS is not None:
    print(f"Traning:    number of arrays:{len(trainIDS)} - number of ids:{sum(len(ids) for ids in trainIDS)}")
    print(f"Validation: number of arrays:{len(validIDS)} - number of ids:{sum(len(ids) for ids in validIDS)}")

    #Analyse the distribution of the legnth of tokens sequences in the ragged/jagged array of tokes
    sectionlengths = np.asarray([len(s) for s in trainIDS],dtype=np.int32)
    plt.hist(sectionlengths[sectionlengths<500], 250, density=True, facecolor='g', alpha=0.75)
    np.histogram(sectionlengths[sectionlengths<1000],50)

    print(f"Lenght of token rags min:{min(sectionlengths)} max:{np.max(sectionlengths)} - median:{np.median(sectionlengths)}")
    print(f"Lenght of token rags mean:{np.mean(sectionlengths)} std:{np.std(sectionlengths)}")

    percent     = np.asarray([ 0, 0.11, 0.2,0.3,0.4,0.5,0.6,0.7, 0.8, 0.9, 1, 1.5, 2, 25,50,75,90,97,99,99.5,99.9,99.99,100]) # %np.arange(101,dtype=np.int)
    percentiles = np.percentile(sectionlengths,percent).astype(np.int)
    print("%  percentile")
    for i in range(len(percent)):print(f"{percent[i]}  {percentiles[i]}")
    minToks = percentiles[np.where( percent==0.11 )[0][0]]
    maxToks = percentiles[np.where( percent==99.99 )[0][0]]
    print(f"\nminToks:{minToks} maxToks:{maxToks}")
    print(f"rags <= minToks:{np.sum(sectionlengths <= minToks)}")
    print(f"rags >= maxToks:{np.sum(sectionlengths >= maxToks)}")

    sectionlengths = trainIDS = validIDS = None
    gc.collect()

In [ ]:
maxUnks,minToks,maxToks = 0,14,500

train_count = valid_count = -1
#train_count = 2.5e9
#valid_count = int(split_ratio * train_count +.5)

%time trainIDS = trainTokenizer.getIds(minToks, maxToks, maxUnks, unk_idx=unk_idx, count=train_count)
%time validIDS = validTokenizer.getIds(minToks, maxToks, maxUnks, unk_idx=unk_idx, count=valid_count)
np.random.seed(1)
np.random.shuffle(trainIDS)
np.random.shuffle(validIDS)

In [ ]:
train_lengths = np.fromiter( (len(s) for s in trainIDS), dtype=np.int16, count=len(trainIDS) )
print(f"Training:   number of arrays:{len(trainIDS)} - number of ids:{sum(len(ids) for ids in trainIDS)}")
print(f"Lenght of training sentences:   min:{np.min(train_lengths)} max:{np.max(train_lengths)} - median:{np.median(train_lengths)}")
plt.hist(train_lengths[train_lengths<maxToks], 100, density=True, facecolor='g', alpha=0.75, label="sentencelength for training data" )
gc.collect()

In [ ]:
valid_lengths = np.fromiter( (len(s) for s in validIDS), dtype=np.int16, count=len(validIDS) )
print(f"Validation: number of arrays:{len(validIDS)} - number of ids:{sum(len(ids) for ids in validIDS)}")
print(f"Lenght of validation sentences: min:{np.min(valid_lengths)} max:{np.max(valid_lengths)} - median:{np.median(valid_lengths)}")
plt.hist(valid_lengths[valid_lengths<maxToks], 100, density=True, facecolor='g', alpha=0.75, label="sentencelength for validation data" )
gc.collect()

In [ ]:
def getNSentencens(sentencelengths, nTokens:int): 
    i,ln_lenghts=0,len(sentencelengths)
    while nTokens>0 and i<ln_lenghts: 
        nTokens -= sentencelengths[i]
        i       += 1
    return i    
#nTokens=int(2.5e9)
nTokens=int(1e8)
print(f"ntokens:{nTokens} i:{getNSentencens(train_lengths, nTokens)}")    
#np.sum(train_lengths[:getNSentencens(train_lengths, nTokens)],dtype=np.int64)
#24922316 

In [ ]:
#from languagemodelloader import *
nTrainToks = int(1e8)
nValidToks = int(0.2*nTrainToks+1)
nTrainSentences, nValidSentences = getNSentencens(train_lengths,nTrainToks), getNSentencens(valid_lengths,nValidToks)
selectedTrainToks = np.sum(train_lengths[:nTrainSentences],dtype=np.int64)
selectedValidToks = np.sum(valid_lengths[:nValidSentences],dtype=np.int64)

print(f"specified training tokens    :{nTrainToks}\nselected training tokens     :{selectedTrainToks}\nselected training sentences  :{nTrainSentences}" )
print(f"\n\nspecified validation tokens  :{nValidToks}\nselected validation tokens   :{selectedValidToks}\nselected validation sentences:{nValidSentences}" )
gc.collect()

In [ ]:
%time dblm = TextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bs=64, bptt=94, num_workers=0, no_check=True)

In [ ]:
dblm.show_batch()

In [ ]:
default_dropout = {'language': np.array([0.25, 0.1, 0.2, 0.02, 0.15]),
                   'classifier': np.array([0.4,0.5,0.05,0.3,0.4])}
dps      = default_dropout["language"]
input_p  = dps[0] 
output_p = dps[1] 
weight_p = dps[2] 
embed_p  = dps[3] 
hidden_p = dps[4] 
out_bias = True

cfg = awd_lstm_lm_config.copy()
cfg['qrnn'],cfg['emb_sz'],cfg['n_hid'],cfg['n_layers'],cfg['tie_weights'], cfg['pad_token'] =\
     False,      256,          1280,            3,           True,               pad_idx 
#     False,      400,         1150,              3,         True,               pad_idx 
cfg['input_p'],cfg['weight_p'],cfg['embed_p'],cfg['hidden_p'],cfg['output_p'], cfg['out_bias']=\
     input_p,       weight_p,       embed_p,       hidden_p,       output_p,   out_bias
drop_mult=.2
%time learn = language_model_learner(dblm, AWD_LSTM,config=cfg, drop_mult=drop_mult, pretrained=False, callback_fns=ShowGraph )
print(cfg)
#callback_fns=PeakMemMetric)

In [ ]:
gc.collect()
%time learn.lr_find()

In [ ]:
learn.recorder.plot(skip_start=0, skip_end=11)
gc.collect()

In [ ]:
from fastai.text import * 
epochs=1
#%time learn.fit_one_cycle(cyc_len=epochs, max_lr=5e-3, moms=(0.99,0.7))
%time learn.fit_one_cycle(cyc_len=epochs, max_lr=2e-3, moms=(0.8,0.7))
# callback_fns=PeakMemMetric

In [ ]:
#remember to update the last validation loss to get your perplexity score
print( f"Perplexity: {exp(3.326004)}") #best: 3.221317
trainTokens={sum(len(s) for s in trainIDS_)}
print( f"Number of tokens in trainingdata:   {sum(len(s) for s in trainIDS_)}")
print( f"Number of tokens in validationdata: {sum(len(s) for s in validIDS_)}")

In [ ]:
learn.save(f'lm-32k-sentencepiece-{epochs}-{len(trainIDS_)}-{len(validIDS_)} dropout{int(round(100*drop_mult))} ')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
losses = learn.recorder.losses
len(losses)

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
#TEXT = "if i knew back then what i know now," #  i would not agree # seen in polval
TEXT = "i liked this movie because "
#TEXT = "anarchy is a political philosophy that"
N_WORDS = 40
N_SENTENCES = 5

In [ ]:
print(("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75, sep="") for _ in range(N_SENTENCES))).replace("▁",""))

In [ ]:
print(("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.25, sep="") for _ in range(N_SENTENCES))).replace("▁",""))

In [ ]:
pd_losses = pd.DataFrame(data=losses)
pd_losses.to_csv(pathTrainValid/"losses_steps.csv", header="losses")